# Projet Intermovie

Ce Notebook a pour but d'analyser un dataset de films dans le but de récupérer plusieurs informations :

- La liste des acteurs par film.

- La liste des films Américains (en gardant leur nom en français) et leur note moyenne.

- Les notes moyennes des différents genres.    

- La note moyenne de chaque acteur par rapport aux films dans lesquels il apparaît.


## Structure des fichiers tsv

- name.basics :         nconst / primaryName / birthYear / deathYear / primaryProfession / knownForTitles
- title.akas :          titleId / ordering / title / region / language / types / attributes / isOriginalTitle
- title.basics :        tconst / titleType / primaryTitle / originalTitle / isAdult / startYear / endYear / runtimeMinutes / genres
- title.principals :    tconst / ordering / nconst / category / job / characters
- title.ratings :       tconst / averageRating / numVotes


# 1- Liste des acteurs par film

## Nettoyage des datasets

- Suppression des colonnes inutiles.
- Suppression des lignes comportant des cellules vides.
- Suppression des lignes entièrement identiques.

On réalise cela pour les fichiers name.basics, title.basics et title.principals.

### Nettoyage du dataset name.basics


In [ ]:
# Extension IPython rechargeant les modules avant que l'utilisateur saisisse du code.
%load_ext autoreload
%autoreload 2

# Import des librairies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df_name_basics = pd.read_table("./data/name.basics.tsv")
df_name_basics.columns
df_name_basics = df_name_basics.drop(['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'], axis = 1)
df_name_basics = df_name_basics.dropna(axis = 0)
df_name_basics = df_name_basics.drop_duplicates()
df_name_basics.describe()

### **Nettoyage du fichier title.basics**

In [ ]:
df_title_basics = pd.read_table("./data/title.basics.tsv")
df_title_basics.columns
df_title_basics = df_title_basics[['tconst', 'titleType', 'originalTitle']][df_title_basics['titleType'].str.contains('movie', regex = True)]
df_title_basics = df_title_basics.drop(['titleType'], axis = 1)
df_title_basics = df_title_basics.dropna(axis = 0)
df_title_basics = df_title_basics.drop_duplicates()
df_title_basics.describe()

### **Nettoyage du fichier title.principals**

In [ ]:
df_title_principals = pd.read_table("./data/title.principals.tsv")
df_title_principals.columns
df_title_principals = df_title_principals[['tconst', 'nconst', 'category']][df_title_principals['category'].str.contains('self|actor|actress', regex = True)]
df_title_principals = df_title_principals.drop(['category'], axis = 1)
df_title_principals = df_title_principals.dropna(axis = 0)
df_title_principals = df_title_principals.drop_duplicates()
df_title_principals.describe()

## Merge des DataFrames + Nettoyage

On fusionne les DataFrames afin d'avoir dans le même DataFrame les noms des acteurs ainsi que les noms des films. On utilise pour cela nos deux clés d'identification : tconst et nconst.

In [33]:
df_fusion = df_title_principals.merge(df_title_basics, how = 'left', on = 'tconst')
df_fusion = df_fusion.merge(df_name_basics, how = 'left', on = 'nconst')
df_fusion = df_fusion.drop(['tconst', 'nconst'], axis = 1)
df_fusion = df_fusion.dropna(axis = 0)
df_final = df_fusion.groupby('originalTitle').agg({'primaryName': ','.join}, axis = 0)
df_final.to_csv('request_1.csv')